# Проект по SQL

Наша компания купила крупный сервис для чтения книг по подписке. В этом проекте мы проанализируем базу данных, в которой содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

Эти данные помогут сформулировать ценностное предложение для нового продукта. В рамках анализа мы выполним следующие задачи:

* посчитаем, сколько книг вышло после 1-го января 2000-го года;
* для каждой книги посчитаем количество обзоров и среднюю оценку;
* определим издательство, которое выпустило наибольшее число книг толще 50-ти страниц;
* определим автора с самой высокой средней оценкой книг (будем учитывать только книги с 50-ю и более оценками);
* посчитаем среднее количество обзоров от пользователей, которые поставили больше 50-ти оценок.

Для начала подключимся к базе данных.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

У нас есть 5 таблиц, выведем первые 5 строк в каждой, и посмотрим, какие данные в них находятся.

In [2]:
query = '''SELECT * FROM books'''
books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


Таблица `books` содержит данные о книгах:

* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

In [4]:
query = '''SELECT * FROM authors'''
authors = pd.io.sql.read_sql(query, con = engine) 
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


Таблица `authors` содержит данные об авторах:

* author_id — идентификатор автора;
* author — имя автора.

In [6]:
query = '''SELECT * FROM publishers'''
publishers = pd.io.sql.read_sql(query, con = engine) 
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


Таблица `publishers` содержит данные об издательствах:

* publisher_id — идентификатор издательства;
* publisher — название издательства.

In [8]:
query = '''SELECT * FROM ratings'''
ratings = pd.io.sql.read_sql(query, con = engine) 
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


Таблица `ratings` содержит данные о пользовательских оценках книг:

* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

In [10]:
query = '''SELECT * FROM reviews'''
reviews = pd.io.sql.read_sql(query, con = engine) 
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [11]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


Таблица `reviews` содержит данные о пользовательских обзорах на книги:

* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя пользователя, написавшего обзор;
* text — текст обзора.

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [12]:
query = '''SELECT COUNT(book_id) AS number_of_books
           FROM books
           WHERE publication_date > '2000-01-01'
           '''
task1 = pd.io.sql.read_sql(query, con = engine) 
task1

,number_of_books
0,819


Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [13]:
query = '''SELECT books.title AS title,
                  books.book_id AS id,
                  AVG(ratings.rating) AS avg_rating,
                  COUNT(DISTINCT reviews.text) AS cnt_reviews
           FROM books
           INNER JOIN ratings ON ratings.book_id = books.book_id
           INNER JOIN reviews ON reviews.book_id = books.book_id
           GROUP BY books.title, books.book_id
           ORDER BY cnt_reviews DESC
           LIMIT 10
           '''
task2 = pd.io.sql.read_sql(query, con = engine) 
task2

,title,id,avg_rating,cnt_reviews
0,Twilight (Twilight #1),948,3.662500,7
1,The Da Vinci Code (Robert Langdon #2),696,3.830508,6
2,Eat Pray Love,207,3.395833,6
3,The Alchemist,627,3.789474,6
4,The Catcher in the Rye,673,3.825581,6
5,The Curious Incident of the Dog in the Night-Time,695,4.081081,6
6,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.414634,6
7,Harry Potter and the Chamber of Secrets (Harry...,299,4.287500,6
8,Outlander (Outlander #1),497,4.125000,6
9,The Book Thief,656,4.264151,6


Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [14]:
query = '''SELECT publisher,
                  COUNT(book_id) AS number_of_books
           FROM publishers p
           JOIN books b ON b.publisher_id = p.publisher_id
           GROUP BY publisher
           HAVING MIN(num_pages) > 50
           ORDER BY number_of_books DESC
           LIMIT 1
           '''
task3 = pd.io.sql.read_sql(query, con = engine) 
task3

,publisher,number_of_books
0,Penguin Books,42


Определим автора с самой высокой средней оценкой книг (учтём только книги с 50 и более оценками).

In [15]:
query = '''SELECT authors.author,
                  top.avg AS average_rating
           FROM authors
           RIGHT JOIN (SELECT author_id,
                              AVG(rating_avg)
                       FROM books
                       LEFT JOIN (SELECT AVG(rating) AS rating_avg, 
                                         book_id 
                                  FROM ratings
                                  GROUP BY book_id) AS ratings ON books.book_id = ratings.book_id
                       WHERE books.book_id IN (SELECT books.book_id
                                               FROM books
                                               LEFT JOIN ratings ON books.book_id = ratings.book_id
                                               GROUP BY books.book_id
                                               HAVING COUNT(rating_id) >= 50)
                       GROUP BY author_id
                       ORDER BY avg DESC
                       LIMIT 1) AS top ON authors.author_id = top.author_id
           '''
task4 = pd.io.sql.read_sql(query, con = engine) 
task4

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [16]:
query = '''SELECT AVG((COUNT)) AS avg_reviews
           FROM
               (SELECT COUNT(text)
                FROM reviews
                FULL OUTER JOIN (SELECT username,
                                        COUNT(rating_id) as cnt
                                 FROM ratings
                                 GROUP BY username) AS cnt_rating
                ON cnt_rating.username = reviews.username
                WHERE cnt > 50
                GROUP BY reviews.username) as count;                                  
                '''
task5 = pd.io.sql.read_sql(query, con = engine) 
task5

,avg_reviews
0,24.333333


## Выводы

По результатам выполнения заданий мы получили следующие выводы:

* после 1 января 2000 года было выпущено 819 книг;
* максимальное количество обзоров у книги Twilight (Twilight #1);
* издательство `Penguin Books` выпустило наибольшее количество книг толще 50-ти страниц - 42 штуки;
* самая высокая средняя оценка у автора `J.K. Rowling/Mary GrandPré`;
* среднее количество обзоров от пользователей, которые поставили больше 50-ти оценок - 24,33.